In [1]:
# importar librerias
import pandas as pd 
from io import StringIO
import re

github:
Si se requiere más información o contexto sobre el código puedes ir a la carpeta 'metacode/' ahí encontrarás 'functions_dictionary.txt' y 'variables_dictionary.txt'. En estos archivos de texto se describe que hace cada función y que guarda cada variable, respectivamente.

In [4]:
# funcion para leer arhivos de texto tabulares con texto de encabezado
def readTxts(location, begin):
    with open(location , 'r') as f:
        # leer por lineas para poder operarlo como lista de lineas
        # begin tiene el numero de linea en donde ya empieza la tabla
        output = f.readlines()[begin:]
        return(output)
    

# funcion para sacar un patron (bnumbers) de cada linea   
def getPatterns(lines, pattern):
    patterns = []
    # buscar en cada linea el patron que concuerda con la ocurrencia de un bnumber y guardarlos en una lista
    for line in lines:
        match = re.search(pattern, line)
        if match:
            extraction = match.group()
            patterns.append(extraction)
        else: patterns.append('')
    return(patterns)


# funcion para pasar de lista de lineas a un string de todo el texto
def makeDF(lines, header_status=None):
    table = ''.join(lines)
    df = pd.read_csv(StringIO(table), delimiter='\t', header=header_status)
    return(df)


# funcion para sustituir una columna con muchos valores (gene synonyms) por uno de interes (bnumber) 
def subsPatternToDF(df, subs, c_receptor):
    df.iloc[:, c_receptor] = subs
    return(df)


In [5]:
# hacer el dataframe para el archivo RegulonDB_geneidentifiers.txt
lines_gene_IDs = readTxts('data/RegulonDB_geneidentifiers.txt', 16)
b_numbers = getPatterns(lines_gene_IDs, r'b\d{4}')
df_gene_IDs = makeDF(lines_gene_IDs)
df_gene_IDs = subsPatternToDF(df_gene_IDs, b_numbers, 5)


In [6]:
# hacer el dataframe para el archivo RegulonDB_NetworkTFGene.txt
lines_TF_gene = readTxts('data/RegulonDB_NetworkTFGene.txt', 37)
df_TF_gene = makeDF(lines_TF_gene, 0)


In [7]:
# leer el archivo de la via y guardar, en otra variable, sus bnumbers asociados
pathway = pd.read_csv('data/pathway_gns_test.txt', delimiter='\t', header=None)
bNumbers_pathway = pathway.iloc[:,1]

In [8]:
# funcion, toma items de una lista y los busca en una columna para entregar el valor de otra columna de la misma fila
def getInterest(df, list_search, c_to_search):
    output_list = []
    for item in list_search:
        # regresa una lista con un True en la posicion en la que coincida la busqueda
        match = df.loc[df.iloc[:, c_to_search] == item]
        # sacar los indices de fila
        # verificar que si exista un match, sino lo hay poner index_in_df vacio
        if not match.empty:
            index_in_df = match.index[0]
        else: index_in_df = None
        # guardar en output_list el valor de la columna de interes
        # en base a si index_in_df tiene indice o vacio llenar segun sea el caso
        if index_in_df != None:
            output_list.append(df.iloc[index_in_df, 1])
        else: output_list.append('unknown')

    return(output_list)

# funcion que transforma de bNumber a TF
def getTFsPathway(initial_geneID):
    genes = getInterest(df_gene_IDs, initial_geneID, 5) # la columna 5 contiene los bNumbers
    TFs = getInterest(df_TF_gene, genes, 4) # la columna 4 contiene el nombre de los genes
    output_df = pd.DataFrame({'bNumber': bNumbers_pathway, 'gene': genes, 'TF': TFs})
    return(output_df)

bN_gene_TF = getTFsPathway(bNumbers_pathway)
print(bN_gene_TF)
    

  bNumber  gene       TF
0   b1270  btuR  unknown
1   b1270  btuR  unknown
2   b1993  cobU      Nac
3   b1991  cobT      Nac
4   b1992  cobS  unknown
5   b1991  cobT      Nac
6   b0638  cobC  unknown


In [16]:
fractions = []
type_subpath = []
tf_most_ocurred = []

# funcion que calcula la fraccion de una subvia controlada por un mismo TF, el mas ocurrente
def getFraction(l, counts, case):
    most_ocurrence = counts[case]
    fractions.append(most_ocurrence/l)
    tf_most_ocurred.append(counts.index[case])

# funcion que prepara todo lo que necesita getFraction para despues llamarla
def callerFraction(current_path, len_sub):
    # .value_counts() saca, de una lista, cuantas veces ocurre cada item unico
    counts = current_path.value_counts()
    # sacar la longitud de la via 
    len_current_path = len(current_path)
    # preguntar de que caso se trata, por lo general no queremos que se tomen en cuenta los 'unkwown's.
    case = 0 if counts.idxmax() != 'unknown' or len(counts) == 1 else 1
    getFraction(len_current_path, counts, case)
    type_subpath.append(len_sub)

# funcion que hace la particion de la via en subvias y pregunta si sus FTs son el mismo
def getSubpathways():
    # guardar en una variable unicamente los TFs, ya en orden, de la via
    TFs = bN_gene_TF['TF']
    len_path = len(TFs)
    # en este primer for se van sacando los tamanos de subvia que puede haber. En este se recorren las sublongitudes
    for len_sub in range(len_path, 1, -1):
        when_stop = len_path-len_sub+1
        final_pos = len_sub
        # en este segundo for se van haciendo las subvias con el tamano acorde al primer for. En este se recorren las 'combinaciones' de cada sublongitud
        for initial_pos in range(0, when_stop):
            subpath = TFs[initial_pos:final_pos]
            callerFraction(subpath, len_sub)
            final_pos+=1
    output = pd.DataFrame({'subpath': type_subpath, 'fraction': fractions, 'TF': tf_most_ocurred})   
    return(output)

# crear un data frame que contenga el tipo de subvia (longitud), la fraccion de la subvia gobernada por un solo TF y el TF en cuestion
subpath_fractions_tf = getSubpathways()
print(subpath_fractions_tf)

    subpath  fraction       TF
0         7  0.428571      Nac
1         6  0.500000      Nac
2         6  0.500000      Nac
3         5  0.400000      Nac
4         5  0.600000      Nac
5         5  0.600000      Nac
6         4  0.500000      Nac
7         4  0.500000      Nac
8         4  0.750000      Nac
9         4  0.500000      Nac
10        3  0.333333      Nac
11        3  0.666667      Nac
12        3  0.666667      Nac
13        3  0.666667      Nac
14        3  0.333333      Nac
15        2  1.000000  unknown
16        2  0.500000      Nac
17        2  1.000000      Nac
18        2  0.500000      Nac
19        2  0.500000      Nac
20        2  0.500000      Nac
